To help Santa, I feel it would be a good approach to first get a good estimate of the weight of each type of gift and once we have the estimate the problem kind of turns itself into kind of Bin packing problem

As all the data is based on certain types of distribution, we can easily find the value points which correspond to certain cumulative probability. For this example, I am taking 85% as the cumulative probability. 

One thing to note would that the weights of books follow Chi-Square distribution and so its PDF depends upon the degrees of freedom, which in this case would 1199, which is huge and most online calculators don't calculate value for it, so for books we are gonna draw 10000 samples to get a good estimate of the 85% percent cumulative value.

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

%matplotlib inline

Number of gifts of each type
 - horse - 1000
 - ball - 1100
 - bike - 500
 - train - 1000
 - coal - 166
 - book - 1200
 - doll - 1000
 - blocks - 1000

In [ ]:
samples = 100000
z_critical = 1.036433 ## For 85% cummulative probability

horse_est_weight = 5 + z_critical*(2)
ball_est_weight = 1 + z_critical*(0.3)
bike_est_weight = 20 + z_critical*(10)
train_est_weight = 10 + z_critical*(5)
coal_est_weight = 47*0.9455  ## Source: http://keisan.casio.com/exec/system/1180573227
doll_est_weight = 7.267 ## Source: http://keisan.casio.com/exec/system/1180573218
block_est_weight = 15.834 ## Source: http://www.math.uah.edu/stat/apps/SpecialCalculator.html
gloves_est_weight = 3 + 0.85
weights = [{'name':'horse', 'weight':horse_est_weight, 'quantity':1000}
          ,{'name':'ball', 'weight':ball_est_weight, 'quantity':1100}
          ,{'name':'bike', 'weight':bike_est_weight, 'quantity':500}
          ,{'name':'train', 'weight':train_est_weight, 'quantity':1000}
          ,{'name':'coal', 'weight':coal_est_weight, 'quantity':166}
          ,{'name':'doll', 'weight':doll_est_weight, 'quantity':1000}
          ,{'name':'blocks', 'weight':block_est_weight, 'quantity':1000}
          ,{'name':'gloves', 'weight':gloves_est_weight, 'quantity':200}]

'''from https://www.kaggle.com/wcukierski/santas-uncertain-bags/plotting-example-gift-weights '''
class book:
    def __init__(self, id):
        self.weight = np.random.chisquare(2,1)[0]
        self.name = 'book'

books = np.histogram([book(x).weight for x in range(samples)], bins = 100)

Now that we have the histogram, let's convert the freqencies of the bin into cummulative relative frequencies.

In [ ]:
def calc_rela_freq_cummulative(hist):
    cummulative = 0
    rel_freq = []
    for val in hist[0]:
        freq = cummulative + float(val)/float(samples)
        cummulative = freq
        rel_freq.append(freq)
    return np.asarray(rel_freq)

books = (calc_rela_freq_cummulative(books), books[1])

In [ ]:
def get_85_percentile(rel_hist):
    for i, x in enumerate(rel_hist[0]):
        if x > 0.845 and x < 0.855:
            return [i, i+1]
    print ('No Element found in that range')
    return [0, 0]

book_est_weight = books[1][get_85_percentile(books)[1]]
print ('For books the value in the 85 percentile is', book_est_weight)

In [ ]:
weights.append({'name':'book', 'weight':book_est_weight, 'quantity': 1200})

In [ ]:
weights

Now that we have a weight estimate for each of the gift, we can try and solve it in a way similar to bin packing problem.

In [ ]:
Weight_per_bag = 50
id_gifts = [0,0,0,0,0,0,0,0,0]
weights = sorted(weights, key=lambda obj: obj['weight'], reverse = True)
bags = 1000
bags_contain = []
for i in range(bags):
    weight_to_fill = Weight_per_bag
    bag_str = ''
    while weight_to_fill > 0:
        if(weight_to_fill < weights[8]['weight']):
            break;
        for j, gift in enumerate(weights):
            if (gift['quantity'] >= 1) and (weight_to_fill - gift['weight'] >= 0):
                bag_str += ' ' + gift['name'] + '_' + str(id_gifts[j]);
                weight_to_fill = weight_to_fill - gift['weight'];
                id_gifts[j] += 1;
                gift['quantity'] -= 1;
    bags_contain.append(bag_str.strip())

In [ ]:
with open("Santa_gifts.csv", 'w') as f:
        f.write("Gifts\n")
        for i in bags_contain:
            f.write( i +'\n')